## SOCRATA QUERY for 311  
Format query by variables.  
By narrowing the query enough, we can avoid paganition in data. 

In [3]:
pip install plotly==5.13.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 18.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
import pandas as pd
import plotly.express as px
pd.set_option( "display.max_columns" , None )

### Generate Query URL  
* Time (> 2022)
* Locations (1 mile)
* descriptors (formatted fro SQL query)  

*What other descriptors to use?*

In [21]:

time_start = '2022-01-01'

# Howard Beach
#lat = 40.6575627
#lon = -73.8409410

# red hook
lat = 40.6744367
lon = -74.0076613

radius = 1609 # 1 mile in meters
descriptors = ['Street Flooding (SJ)','Catch Basin Clogged/Flooding (Use Comments) (SC)', 'Sewer Backup (Use Comments) (SA)', 'Failure To Retain Water/Improper Drainage- (LL103/89)','Manhole Overflow (Use Comments) (SA1)', 'Snow/Ice'] 

def get_data(time_start, lat, lon , descriptors):

    #Convert descriptors to multiple category searches
    cats = "({})".format( " ".join( [f"descriptor='{i}' OR" for i in descriptors] )[:-3] )
    base_url = f"https://data.cityofnewyork.us/resource/erm2-nwe9.json?$where=created_date>='{time_start}' AND within_circle(location, {lat}, {lon}, {radius}) AND {cats}"

    print( base_url )

    response = requests.get( base_url )

    df = pd.DataFrame().from_records( response.json() )

    print( "{} rows are found".format( df.shape[0] ) )
    
    return df

df = get_data( time_start , lat, lon , descriptors )
df.head()

https://data.cityofnewyork.us/resource/erm2-nwe9.json?$where=created_date>='2022-01-01' AND within_circle(location, 40.6744367, -74.0076613, 1609) AND (descriptor='Street Flooding (SJ)' OR descriptor='Catch Basin Clogged/Flooding (Use Comments) (SC)' OR descriptor='Sewer Backup (Use Comments) (SA)' OR descriptor='Failure To Retain Water/Improper Drainage- (LL103/89)' OR descriptor='Manhole Overflow (Use Comments) (SA1)' OR descriptor='Snow/Ice')
176 rows are found


,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,address_type,city,status,community_board,bbl,borough,x_coordinate_state_plane,y_coordinate_state_plane,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location,intersection_street_1,intersection_street_2,closed_date,resolution_description,resolution_action_updated_date
0,56595321,2023-01-22T23:32:00.000,DEP,Department of Environmental Protection,Sewer,Catch Basin Clogged/Flooding (Use Comments) (SC),11215,561 4 AVENUE,4 AVENUE,16 ST,PROSPECT AVE,ADDRESS,BROOKLYN,Open,07 BROOKLYN,3010520011,BROOKLYN,986384,181932,PHONE,Unspecified,BROOKLYN,40.666037786241475,-73.99230775001512,"{'latitude': '40.666037786241475', 'longitude'...",NaN,NaN,NaN,NaN,NaN
1,56594564,2023-01-22T21:26:00.000,DEP,Department of Environmental Protection,Sewer,Catch Basin Clogged/Flooding (Use Comments) (SC),11231,NaN,NaN,NaN,NaN,INTERSECTION,BROOKLYN,Open,06 BROOKLYN,NaN,BROOKLYN,981698,187564,ONLINE,Unspecified,BROOKLYN,40.681496249864225,-74.00920111176733,"{'latitude': '40.681496249864225', 'longitude'...",COMMERCE STREET,IMLAY STREET,NaN,NaN,NaN
2,56572579,2023-01-19T15:20:00.000,DEP,Department of Environmental Protection,Sewer,Sewer Backup (Use Comments) (SA),11231,128 UNION STREET,UNION STREET,COLUMBIA ST,HICKS ST,ADDRESS,BROOKLYN,Closed,06 BROOKLYN,3003420016,BROOKLYN,983596,188548,ONLINE,Unspecified,BROOKLYN,40.6841974494735,-74.00235806067457,"{'latitude': '40.6841974494735', 'longitude': ...",NaN,NaN,2023-01-19T20:20:00.000,The Department of Environmental Protection has...,2023-01-19T20:20:00.000
3,56567558,2023-01-19T15:08:00.000,DEP,Department of Environmental Protection,Sewer,Sewer Backup (Use Comments) (SA),11231,126 UNION STREET,UNION STREET,COLUMBIA ST,HICKS ST,ADDRESS,BROOKLYN,Closed,06 BROOKLYN,3003420015,BROOKLYN,983582,188553,ONLINE,Unspecified,BROOKLYN,40.68421117228693,-74.00240853953329,"{'latitude': '40.68421117228693', 'longitude':...",NaN,NaN,2023-01-19T20:20:00.000,The Department of Environmental Protection has...,2023-01-19T20:20:00.000
4,56571594,2023-01-19T08:20:00.000,DEP,Department of Environmental Protection,Sewer,Sewer Backup (Use Comments) (SA),11231,288 COURT STREET,COURT STREET,DOUGLASS ST,DE GRAW ST,ADDRESS,BROOKLYN,Closed,06 BROOKLYN,3003260056,BROOKLYN,985635,188561,PHONE,Unspecified,BROOKLYN,40.68423304730939,-73.9950062449861,"{'latitude': '40.68423304730939', 'longitude':...",NaN,NaN,2023-01-19T10:15:00.000,The Department of Environmental Protection has...,2023-01-19T10:15:00.000


In [22]:
df['created_date'] = pd.to_datetime( df['created_date'] )
pt = pd.pivot_table(
    data = df,
    index = 'descriptor',
    columns = pd.Grouper(key='created_date' , freq = '1W'),
    aggfunc = len ,
    values='unique_key'
).fillna(0)

pt

created_date,2022-01-02,2022-01-16,2022-01-23,2022-02-06,2022-02-13,2022-02-20,2022-02-27,2022-03-13,2022-03-20,2022-03-27,2022-04-03,2022-04-10,2022-04-17,2022-05-01,2022-05-08,2022-05-15,2022-05-22,2022-05-29,2022-06-05,2022-06-12,2022-06-19,2022-06-26,2022-07-03,2022-07-10,2022-07-17,2022-07-24,2022-07-31,2022-08-14,2022-08-21,2022-08-28,2022-09-04,2022-09-11,2022-09-18,2022-09-25,2022-10-02,2022-10-09,2022-10-16,2022-10-23,2022-10-30,2022-11-06,2022-11-13,2022-11-20,2022-11-27,2022-12-04,2022-12-11,2022-12-18,2022-12-25,2023-01-01,2023-01-08,2023-01-15,2023-01-22
descriptor,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Catch Basin Clogged/Flooding (Use Comments) (SC),0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
Failure To Retain Water/Improper Drainage- (LL103/89),0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Manhole Overflow (Use Comments) (SA1),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Sewer Backup (Use Comments) (SA),1.0,2.0,1.0,3.0,2.0,10.0,1.0,2.0,3.0,1.0,1.0,3.0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,3.0,2.0,1.0,0.0,1.0,1.0,17.0,7.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,3.0
Street Flooding (SJ),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,6.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0


In [7]:
#pip install -U kaleido


  Using cached kaleido-0.2.1-py2.py3-none-macosx_10_11_x86_64.whl (85.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [23]:
import plotly.express as px

fig = px.imshow(
    pt,
    color_continuous_scale='OrRd'
)

#fig.write_html("plotly_charts/heatmap_redhook.html" , include_plotlyjs='directory' , div_id='heatmap' ,full_html=False ) 
#fig.write_html("plotly_charts/heatmap_howardbeach.html" , include_plotlyjs='directory' , div_id='heatmap' ,full_html=False ) 
fig.write_html("plotly_charts/heatmap_redhook.html" , include_plotlyjs='directory' , div_id='heatmap' ,full_html=False ) 
#fig.write_image("plotly_charts/heatmap_howardbeach.svg")
fig.write_image("plotly_charts/heatmap_redhook.svg")

In [11]:

gr = df.groupby("agency").size()

fig = px.bar(
    data_frame=gr
)
#fig.write_html("plotly_charts/barchart_redhook.html" , include_plotlyjs='directory' , div_id='bar-chart',full_html=False )
fig.write_html("plotly_charts/barchart_howardbeach.html" , include_plotlyjs='directory' , div_id='bar-chart',full_html=False )
fig.write_image("plotly_charts/barchart_howardbeach.svg")



In [24]:
Red Hook
gr = df.groupby("agency").size()

fig = px.bar(
    data_frame=gr
)
#fig.write_html("plotly_charts/barchart_redhook.html" , include_plotlyjs='directory' , div_id='bar-chart',full_html=False )
fig.write_html("plotly_charts/barchart_redhook.html" , include_plotlyjs='directory' , div_id='bar-chart',full_html=False )
fig.write_image("plotly_charts/barchart_redhook.svg")

SyntaxError: invalid syntax (765269287.py, line 1)